<a href="https://colab.research.google.com/github/mjs1995/study/blob/main/study/tree/main/spark/pyspark_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pyspark
- https://sparkbyexamples.com/pyspark/pyspark-when-otherwise/

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 35 kB/s 
     |████████████████████████████████| 198 kB 46.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=e27fd9ac7b9d9ce8bde2986cea67ce179ae35a47870bce676ffcd24c7f48ca14
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
                    .master('local[*]')\
                    .appName('hello_world_app')\
                    .getOrCreate()

In [ ]:
values = [("2021-08-03","a", 23, 20), ("2021-09-03","b", 45,10), ("2021-10-03","c", 10,20), ("2021-11-03","d", 60,20), ("2021-08-23","e", -99,-99), ("2021-09-13","f", 2,10), ("2021-11-23","g", 25,20), ("2021-11-23","h", 40,10), ("2021-09-05","j", 33,-99)]
df = spark.createDataFrame(values, ["date","name", "ages",'gender'])

In [ ]:
df.show()

+----------+----+----+------+
|      date|name|ages|gender|
+----------+----+----+------+
|2021-08-03|   a|  23|    20|
|2021-09-03|   b|  45|    10|
|2021-10-03|   c|  10|    20|
|2021-11-03|   d|  60|    20|
|2021-08-23|   e| -99|   -99|
|2021-09-13|   f|   2|    10|
|2021-11-23|   g|  25|    20|
|2021-11-23|   h|  40|    10|
|2021-09-05|   j|  33|   -99|
+----------+----+----+------+



In [ ]:
# 나이 구분 
from pyspark.sql.functions import udf
from pyspark.sql.types import *

def categorizer(ages):
  if ages < 10:
    return "정보없음"
  elif ages < 20:
    return "10대"
  elif ages < 30:
    return "20대"
  elif ages < 40:
    return "30대"
  elif ages >= 40:
    return "40대 이상"  
  else: 
    return "정보없음"


bucket_udf = udf(categorizer, StringType() )
bucketed = df.withColumn("bucket", bucket_udf("ages"))

In [ ]:
bucketed.show()

+----------+----+----+------+---------+
|      date|name|ages|gender|   bucket|
+----------+----+----+------+---------+
|2021-08-03|   a|  23|    20|     20대|
|2021-09-03|   b|  45|    10|40대 이상|
|2021-10-03|   c|  10|    20|     10대|
|2021-11-03|   d|  60|    20|40대 이상|
|2021-08-23|   e| -99|   -99| 정보없음|
|2021-09-13|   f|   2|    10| 정보없음|
|2021-11-23|   g|  25|    20|     20대|
|2021-11-23|   h|  40|    10|40대 이상|
|2021-09-05|   j|  33|   -99|     30대|
+----------+----+----+------+---------+



In [ ]:
import pyspark.sql.functions as f

bucketed2 = bucketed.withColumn("new_gender", f.when(bucketed.gender == 20,"여")
    .when(df.gender == 10,"남")
     #.when(df.gender.isNull() ,"")
    .otherwise('정보없음'))

In [ ]:
bucketed2.show()

+----------+----+----+------+---------+----------+
|      date|name|ages|gender|   bucket|new_gender|
+----------+----+----+------+---------+----------+
|2021-08-03|   a|  23|    20|     20대|        여|
|2021-09-03|   b|  45|    10|40대 이상|        남|
|2021-10-03|   c|  10|    20|     10대|        여|
|2021-11-03|   d|  60|    20|40대 이상|        여|
|2021-08-23|   e| -99|   -99| 정보없음|  정보없음|
|2021-09-13|   f|   2|    10| 정보없음|        남|
|2021-11-23|   g|  25|    20|     20대|        여|
|2021-11-23|   h|  40|    10|40대 이상|        남|
|2021-09-05|   j|  33|   -99|     30대|  정보없음|
+----------+----+----+------+---------+----------+



In [ ]:
bucketed2

DataFrame[name: string, ages: bigint, gender: bigint, bucket: string, new_gender: string]

In [ ]:
#bucketed3=bucketed2.groupBy('bucket','new_gender').agg(f.countDistinct('name').alias('회원 수')).orderBy('bucket','new_gender')
bucketed3=bucketed2.groupBy('new_gender','bucket').agg(f.countDistinct('name').alias('회원 수')).orderBy('new_gender','bucket')

In [ ]:
bucketed3.show()

+----------+---------+-------+
|new_gender|   bucket|회원 수|
+----------+---------+-------+
|        남|40대 이상|      2|
|        남| 정보없음|      1|
|        여|     10대|      1|
|        여|     20대|      2|
|        여|40대 이상|      1|
|  정보없음|     30대|      1|
|  정보없음| 정보없음|      1|
+----------+---------+-------+



In [ ]:
val = [("2021-08-03","MEM100000000185801"), ("2021-09-03","MEM100000000330231"), ("2021-10-03","MEM100000000099236")]
data = spark.createDataFrame(val, ["최근2개월 로그인 날짜","회원ID"])
data.show()

+---------------------+------------------+
|최근2개월 로그인 날짜|            회원ID|
+---------------------+------------------+
|           2021-08-03|MEM100000000185801|
|           2021-09-03|MEM100000000330231|
|           2021-10-03|MEM100000000099236|
+---------------------+------------------+



In [ ]:
val = [("MEM100000000185801", 23, 20), ("MEM100000000330231", 45,10), ("MEM100000000099236", 10,20)]
data = spark.createDataFrame(val, ["회원ID", "나이",'성별'])
data.show()

+------------------+----+----+
|            회원ID|나이|성별|
+------------------+----+----+
|MEM100000000185801|  23|  20|
|MEM100000000330231|  45|  10|
|MEM100000000099236|  10|  20|
+------------------+----+----+



# PySpark DataFrame

## Create DataFrame

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window
import pyspark.sql.functions as f

In [ ]:
columns = ["language","users_count"]
data = [("Java", "20000"), ("Python", "100000"), ("Scala", "3000")]

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
rdd = spark.sparkContext.parallelize(data)

In [ ]:
dfFromRDD1 = rdd.toDF()
dfFromRDD1.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)



In [ ]:
dfFromRDD1.show()

+------+------+
|    _1|    _2|
+------+------+
|  Java| 20000|
|Python|100000|
| Scala|  3000|
+------+------+



In [ ]:
columns = ["language","users_count"]
dfFromRDD1 = rdd.toDF(columns)
dfFromRDD1.printSchema()

root
 |-- language: string (nullable = true)
 |-- users_count: string (nullable = true)



In [ ]:
dfFromRDD2 = spark.createDataFrame(rdd).toDF(*columns)

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
data2 = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data2,schema=schema)
df.printSchema()
df.show(truncate=False) # 전체 칼럼 보여줄지 말지 

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



## Create an Empty DataFrame & RDD

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

#Creates Empty RDD
emptyRDD = spark.sparkContext.emptyRDD()
print(emptyRDD)

EmptyRDD[147] at emptyRDD at NativeMethodAccessorImpl.java:0


In [ ]:
rdd2= spark.sparkContext.parallelize([])
print(rdd2)

ParallelCollectionRDD[148] at readRDDFromFile at PythonRDD.scala:274


In [ ]:
from pyspark.sql.types import StructType,StructField, StringType
schema = StructType([
  StructField('firstname', StringType(), True),
  StructField('middlename', StringType(), True),
  StructField('lastname', StringType(), True)
  ])

In [ ]:
df = spark.createDataFrame(emptyRDD,schema)
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)



In [ ]:
df1 = emptyRDD.toDF(schema)
df1.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)



In [ ]:
df1.show()

+---------+----------+--------+
|firstname|middlename|lastname|
+---------+----------+--------+
+---------+----------+--------+



In [ ]:
df2 = spark.createDataFrame([], schema)
df2.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)



In [ ]:
df3 = spark.createDataFrame([], StructType([]))
df3.printSchema()

root

